In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
import time
import warnings

from tqdm import tqdm

# Suppress unnecessary FutureWarning
warnings.filterwarnings('ignore', category=FutureWarning)

In [12]:
docs = pd.read_csv('/kaggle/input/lab-nir-2024/lab_docs.csv')
topics = pd.read_csv('/kaggle/input/lab-nir-2024/lab_topics.csv')
qrels = pd.read_csv('/kaggle/input/lab-nir-2024/lab_qrels.csv')

In [3]:
!curl -s "https://get.sdkman.io" | bash && source "$HOME/.sdkman/bin/sdkman-init.sh" && sdk install java 11.0.22-amzn < /dev/null


                                -+syyyyyyys:
                            `/yho:`       -yd.
                         `/yh/`             +m.
                       .oho.                 hy                          .`
                     .sh/`                   :N`                `-/o`  `+dyyo:.
                   .yh:`                     `M-          `-/osysoym  :hs` `-+sys:      hhyssssssssy+
                 .sh:`                       `N:          ms/-``  yy.yh-      -hy.    `.N-````````+N.
               `od/`                         `N-       -/oM-      ddd+`     `sd:     hNNm        -N:
              :do`                           .M.       dMMM-     `ms.      /d+`     `NMMs       `do
            .yy-                             :N`    ```mMMM.      -      -hy.       /MMM:       yh
          `+d+`           `:/oo/`       `-/osyh/ossssssdNMM`           .sh:         yMMN`      /m.
         -dh-           :ymNMMMMy  `-/shmNm-`:N/-.``   `.sN            /N-         `NMMy      .m/
  

In [4]:
pip install python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
%env JAVA_HOME=/root/.sdkman/candidates/java/current
import pyterrier as pt
if not pt.started():
    pt.init()

env: JAVA_HOME=/root/.sdkman/candidates/java/current
terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [6]:
start_time = time.time()

indexer = pt.DFIndexer("./indexes/default", overwrite=True)
index_ref = indexer.index(docs["text"], docs["docno"].astype(str))
index_ref.toString()
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

end_time = time.time()
print(end_time - start_time)
len((index.getLexicon()))

Number of documents: 2453
Number of terms: 23693
Number of postings: 208487
Number of fields: 0
Number of tokens: 273373
Field names: []
Positions:   false

5.137542486190796


23693

In [ ]:
search_times = []

# randomly selected 1000 items
ix_range = random.sample(range(0, 4023296), 1000)

# timimg
for ix, kv in enumerate(index.getLexicon()):
    start_time = time.time()
    if ix in ix_range:
        pointer = index.getLexicon()[kv.getKey()]
    end_time = time.time()
    search_time = end_time - start_time
    search_times.append(search_time)
    
print(sum(search_times)/1000)

In [ ]:
# Stopwords removal
start_time = time.time()

indexer = pt.DFIndexer("./indexes/stopwords", overwrite=True)
indexer.setProperty("termpipelines", "Stopwords")
index_ref = indexer.index(docs["text"], docs["docno"].astype(str))
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

end_time = time.time()
print(end_time - start_time)
len((index.getLexicon()))

In [ ]:
# Stemmer
start_time = time.time()

indexer = pt.DFIndexer("./indexes/stemmer", overwrite=True)
indexer.setProperty("termpipelines", "PorterStemmer")
index_ref = indexer.index(docs["text"], docs["docno"].astype(str))
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

end_time = time.time()
print(end_time - start_time)
len((index.getLexicon()))

In [ ]:
# No pre-processing
start_time = time.time()

indexer = pt.DFIndexer("./indexes/none", overwrite=True)
indexer.setProperty("termpipelines", "")
index_ref = indexer.index(docs["text"], docs["docno"].astype(str))
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

end_time = time.time()
print(end_time - start_time)
len((index.getLexicon()))

In [11]:
BM25 = pt.BatchRetrieve(index, wmodel="BM25")

In [16]:
from pyterrier.measures import *
K = 5

pt.Experiment(
    retr_systems=[BM25],
    names = ["BM25"],
    topics=topics,
    qrels=qrels,
    eval_metrics=[NDCG@10, 'map']
)

,name,nDCG@10,map
0,BM25,0.842503,0.628454


In [ ]:
print(topics.shape)
topics.sample(5)

In [ ]:
qrels.sample(5)

In [ ]:
qrels["label"].dtype

In [14]:
topics["qid"] = topics["qid"].astype(str)
qrels["qid"] = qrels["qid"].astype(str)
qrels["docno"] = qrels["docno"].astype(str)

In [ ]:
qrels["qid"].isna().any()